In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases
from decimal import Decimal
import datetime as dt

In [4]:
!pip install decimal

ERROR: Could not find a version that satisfies the requirement decimal (from versions: none)
ERROR: No matching distribution found for decimal


In [5]:
# pip install lifetimes
# pip install openpyxl

In [6]:
online_retail = pd.read_excel(
    "data/raw/online_retail.xlsx", sheet_name="Year 2009-2010"
)
print(online_retail.shape)
online_retail.head()

(525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
online_retail.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [8]:
online_retail.dropna(inplace=True)
print(online_retail.isna().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64


In [9]:
online_retail.loc[online_retail["Quantity"] < 0].shape

(9839, 8)

In [10]:
online_retail = online_retail[~online_retail["Quantity"] < 0]
online_retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407695 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      407695 non-null  object        
 1   StockCode    407695 non-null  object        
 2   Description  407695 non-null  object        
 3   Quantity     407695 non-null  int64         
 4   InvoiceDate  407695 non-null  datetime64[ns]
 5   Price        407695 non-null  float64       
 6   Customer ID  407695 non-null  float64       
 7   Country      407695 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.0+ MB


In [11]:
online_retail["Invoice"] = online_retail["Invoice"].astype(str)
online_retail["StockCode"] = online_retail["StockCode"].astype(str)
# online_retail['InvoiceDate'] = pd.to_datetime(online_retail['InvoiceDate'])
online_retail["Customer ID"] = online_retail["Customer ID"].astype(str)
online_retail["Description"] = online_retail["Description"].astype(str)
online_retail["Country"] = online_retail["Country"].astype(str)
online_retail["Price"] = online_retail["Price"].astype(float)

In [12]:
online_retail.shape

(407695, 8)

In [13]:
online_retail["Monetary"] = online_retail["Price"] * online_retail["Quantity"]
online_retail["Monetary"] = online_retail["Monetary"].astype(float)

In [14]:
df_rfmt = summary_data_from_transaction_data(
    transactions=online_retail,
    customer_id_col="Customer ID",
    datetime_col="InvoiceDate",
    monetary_value_col="Monetary",
)
print(df_rfmt.head())
print(df_rfmt.shape)

             frequency  recency      T  monetary_value
Customer ID                                           
12346.0            6.0    196.0  360.0       47.143333
12347.0            1.0     37.0   39.0      711.790000
12348.0            0.0      0.0   73.0        0.000000
12349.0            2.0    182.0  224.0      801.310000
12351.0            0.0      0.0   10.0        0.000000
(4314, 4)


In [15]:
online_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Monetary
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0
